In [ ]:
%%capture
!pip install sentence-transformers

In [ ]:
import pandas as pd
with open('cleaned_content.csv', 'r', encoding='utf-8') as f:
    content = pd.read_csv(f)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
content.dropna(inplace=True)
content = content.sample(frac=1).reset_index(drop=True)
content.reset_index(inplace=True, drop=True)

In [ ]:
content

In [ ]:
### thay enter bằng space, các kí tự . . . . . bị thay bằng space, thay thế n-spaces bằng duy nhất 1 space

import re
def strip_context(text):
    text = text.replace('\n', ' ')
    text = text.replace('"', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

documents = [{'id_book':row['id_book'],'name_chapter':row['name_chapter'],'document':strip_context(row['clean_chapter'])} for _, row in content.iterrows()]
print(len(documents))

In [ ]:
### Tách một chương truyện thành từng câu, sau đó nối các câu thành từng đoạn, mỗi đoạn không vượt quá 254 từ

import re

all_passage_dict = []
for i, row in content.iterrows():
    chapter = documents[i]['document']
    passage = ""
    num_word = 0
    for word in chapter.split():
        if num_word <= 254:
            passage = passage + ' ' + word
            num_word += 1
        else:
            all_passage_dict.append({'id_book':row['id_book'], 'name_chapter':row['name_chapter'], 'passage':passage})
            passage = ""
            num_word = 0
    # nếu đoạn cuối cùng trong 1 chương đủ dài thì cũng lấy
    if len(passage.split()) > 70:
        all_passage_dict.append({'id_book':row['id_book'], 'name_chapter':row['name_chapter'], 'passage':passage})

In [ ]:
len(all_passage_dict)

In [ ]:
# tách đoạn thành công nhé
for item in all_passage_dict:
    if len(item['passage'].split()) < 60:
        print("error")

# Chọn ra 200.000 passage để học unsupervised

In [ ]:
import random
random.shuffle(all_passage_dict)
all_paraghraphs = []
for i in range(0, len(all_passage_dict)):
    all_paraghraphs.append(all_passage_dict[i]['passage'])

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, datasets, losses
from torch.utils.data import DataLoader

# dataset class with noise functionality built-in
train_dataset = datasets.DenoisingAutoEncoderDataset(all_paraghraphs)

# we use a dataloader as usual
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)

In [ ]:
from sentence_transformers import SentenceTransformer, models

model = SentenceTransformer('keepitreal/vietnamese-sbert')
model

In [ ]:
from sentence_transformers.losses import DenoisingAutoEncoderLoss

train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path='keepitreal/vietnamese-sbert', tie_encoder_decoder=True)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('model-v4')

# Lập chỉ mục cho tất cả đoạn văn

In [ ]:
x = [passage['passage'] for passage in all_passage_dict]

In [ ]:
len(x)

In [ ]:
count = 0
for sub_x in x:
    num_words = len(sub_x.split())
    if num_words <= 70:
        count += 1
print(count)

In [ ]:
corpus_embeddings = model.encode(x[:], convert_to_tensor=True, show_progress_bar=True)
corpus_embeddings

In [ ]:
embeddings = [x.tolist() for x in corpus_embeddings]

In [ ]:
len(embeddings)

In [ ]:
id_book = []
chapter = []
all_passages = []
for passage in all_passage_dict[:]:
    id_book.append(passage['id_book'])
    chapter.append(passage['name_chapter'])
    all_passages.append(passage['passage'])

In [ ]:
database = pd.DataFrame()
database['id_book'] = id_book
database['name_chapter'] = chapter
database['passages'] = all_passages
database['embedding'] = embeddings

In [ ]:
database

In [ ]:
database.to_parquet('indexed_book_model_v4.parquet.gzip', compression='gzip')